In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
%matplotlib inline
pd.set_option('display.max_columns', 500)

In [26]:
advworkscusts = pd.read_csv('AdvWorksCusts.csv')
aw_avemonthspend = pd.read_csv('AW_AveMonthSpend.csv')
aw_test = pd.read_csv('AW_test.csv')

In [27]:
advworkscusts['BirthDate'] = pd.to_datetime(advworkscusts['BirthDate'])
advworkscusts['age'] = 1998 - advworkscusts['BirthDate'].dt.year

aw_test['BirthDate'] = pd.to_datetime(aw_test['BirthDate'])
aw_test['age'] = 1998 - aw_test['BirthDate'].dt.year

In [28]:
advworkscusts.drop(['CustomerID', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix', 'AddressLine1', 'AddressLine2'], axis=1, inplace=True)
advworkscusts.drop(['City', 'StateProvinceName', 'PostalCode', 'PhoneNumber', 'BirthDate'], axis=1, inplace=True)

In [30]:
aw_test.drop(['CustomerID', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix', 'AddressLine1', 'AddressLine2'], axis=1, inplace=True)
aw_test.drop(['City', 'StateProvinceName', 'PostalCode', 'PhoneNumber', 'BirthDate'], axis=1, inplace=True)

In [32]:
advworkscusts = pd.get_dummies(advworkscusts)
aw_test = pd.get_dummies(aw_test)

In [34]:
train_X, val_X, train_y, val_y = (train_test_split(advworkscusts, aw_avemonthspend['AveMonthSpend'], test_size=0.2, random_state=2019))

In [35]:
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV

In [36]:
parameters= {'max_depth':[3, 4, 5, 6, 7, 8, 9, 10], 'learning_rate':[0.1, 0.01, 0.001, 0.0001, 0.00001], 'n_estimators':[100, 300, 500, 1000]}
model = XGBRegressor()
model.fit(advworkscusts, aw_avemonthspend['AveMonthSpend'])
best_model = RandomizedSearchCV(model, parameters, cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=4, n_iter=50)
best_model.fit(advworkscusts, aw_avemonthspend['AveMonthSpend'])
best_model.best_estimator_

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  2.6min
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed: 18.7min
[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed: 24.3min finished


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.01, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [37]:
eval_set =  [(val_X, val_y)]
model = XGBRegressor(learning_rate=0.1, max_depth=5, n_estimators=100)
model.fit(train_X, train_y, early_stopping_rounds=10, eval_metric="rmse",
          eval_set=eval_set, verbose=True)

[0]	validation_0-rmse:69.5228
Will train until validation_0-rmse hasn't improved in 10 rounds.
[1]	validation_0-rmse:62.6405
[2]	validation_0-rmse:56.4563
[3]	validation_0-rmse:50.8863
[4]	validation_0-rmse:45.8759
[5]	validation_0-rmse:41.3688
[6]	validation_0-rmse:37.3121
[7]	validation_0-rmse:33.6628
[8]	validation_0-rmse:30.3878
[9]	validation_0-rmse:27.431
[10]	validation_0-rmse:24.7761
[11]	validation_0-rmse:22.3904
[12]	validation_0-rmse:20.2481
[13]	validation_0-rmse:18.3216
[14]	validation_0-rmse:16.5904
[15]	validation_0-rmse:15.0354
[16]	validation_0-rmse:13.6411
[17]	validation_0-rmse:12.3941
[18]	validation_0-rmse:11.2752
[19]	validation_0-rmse:10.2717
[20]	validation_0-rmse:9.37638
[21]	validation_0-rmse:8.58094
[22]	validation_0-rmse:7.87446
[23]	validation_0-rmse:7.24696
[24]	validation_0-rmse:6.6963
[25]	validation_0-rmse:6.20144
[26]	validation_0-rmse:5.76863
[27]	validation_0-rmse:5.39022
[28]	validation_0-rmse:5.06181
[29]	validation_0-rmse:4.77464
[30]	validation_0

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [38]:
test_y = model.predict(aw_test)

In [39]:
aw_test = pd.read_csv('AW_test.csv')

In [40]:
result = pd.DataFrame(data=list(zip(aw_test['CustomerID'], test_y)), columns=['CustomerID', 'AveMonthSpend'])

In [41]:
result.to_csv('result.csv', index=False)